In [12]:
import pandas as pd
import numpy as np
import os, joblib
from sentence_transformers import SentenceTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

In [2]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [3]:
# Load dataset
df = pd.read_csv('input_train_data.csv')  # columns: answer, category, subcategory

# Load model
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\USER\anaconda3\envs\textclass\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Encode categories
le = LabelEncoder()
y = le.fit_transform(df['category'])

In [5]:
# Get embeddings
X = model.encode(df['answer'].astype(str).fillna('').tolist(), show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# Train One-vs-Rest classifier
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))
clf.fit(X, y)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=1000))

In [8]:
# Predict
y_pred         = clf.predict(X)
y_pred_labels  = le.inverse_transform(y_pred)
y_probs        = clf.predict_proba(X)
confidence     = np.max(y_probs, axis=1)

In [ ]:
# Add predictions to DataFrame
df['predicted_category'] = y_pred_labels
df['confidence_score']   = confidence

In [9]:
# Evaluate
print("Accuracy:", accuracy_score(df['category'], y_pred_labels))
print(classification_report(df['category'], y_pred_labels, digits=4))

Accuracy: 1.0
                   precision    recall  f1-score   support

    Career Growth     1.0000    1.0000    1.0000        17
     Compensation     1.0000    1.0000    1.0000        13
         Job Role     1.0000    1.0000    1.0000        12
       Management     1.0000    1.0000    1.0000        16
 Work Environment     1.0000    1.0000    1.0000        20
Work-Life Balance     1.0000    1.0000    1.0000        12
Workplace Culture     1.0000    1.0000    1.0000        10

         accuracy                         1.0000       100
        macro avg     1.0000    1.0000    1.0000       100
     weighted avg     1.0000    1.0000    1.0000       100



In [9]:
# Add predictions to DataFrame
df['predicted_category'] = y_pred_labels
df.to_csv('answers_with_predicted_category_supervised.csv', index=False)

In [13]:
# Save all artifacts to emb/
np.save('emb/category_embeddings.npy', X)
joblib.dump(clf,         'emb/classifier.pkl')
joblib.dump(le,          'emb/label_encoder.pkl')
model.save('emb/sentence_model')